In [44]:
#Importación de librearías necesarias
import pandas as pd
from datetime import datetime
import socket

In [45]:
socket.gethostname()

'LAPTOP-PUSGG08B'

In [22]:
#Path general de archivos
if socket.gethostname()=='LAPTOP-PUSGG08B': #Ip de la laptop
    ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/Data/TFM/"
elif socket.gethostname()=='LAPTOP-PUSGG08B': #Ip del working
    ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/Data/TFM/"
# Ruta del archivo de pensionistas de vejez
ruta_vj = ruta + 'POB_VEJ_CD656.dsv'
# Ruta del archivo de historia laboral de pensionistas
ruta_afi = ruta + 'APORTES_CD656.dsv'

In [19]:
vej = pd.read_csv(ruta_vj, delimiter='\t', encoding='utf-8', decimal='.')
vej['FECHA_NACIMIENTO'] = pd.to_datetime(vej['FECHA_NACIMIENTO'], format='%d/%m/%Y')
vej['FECHA_MUERTE'] = pd.to_datetime(vej['FECHA_MUERTE'], format='%d/%m/%Y')
vej['FECHA_DERECHO'] = pd.to_datetime(vej['FECHA_DERECHO'], format='%d/%m/%Y')
vej['RANGO_INI_5MEJ'] = pd.to_datetime(vej['RANGO_INI_5MEJ'], format='%d/%m/%Y')
vej['RANGO_FIN_5MEJ'] = pd.to_datetime(vej['RANGO_FIN_5MEJ'], format='%d/%m/%Y')

In [21]:
#Datos de vejez
vej

,CEDULA_COD,SEXO,FECHA_NACIMIENTO,FECHA_MUERTE,FECHA_DERECHO,NUM_IMPOSICIONES,COEFICIENTE_TEO,COEFICIENTE_REAL,BASE_AP_TEO,BASE_AP_REAL,PENSION,TIPO_PRESTACION,RANGO_INI_5MEJ,RANGO_FIN_5MEJ
0,114,H,1921-07-12,NaT,1994-07-01,429,0.500000,0.500000,0.00,0.00,28.09,VEJEZ,2004-10-31,1991-05-01
1,73969,H,1934-05-29,NaT,1997-01-01,481,1.001042,1.001042,0.00,0.00,35.06,VEJEZ,1996-12-31,1992-01-01
2,44450,H,1936-02-18,NaT,1994-06-24,436,0.841833,0.841833,1770.94,1770.94,363.74,VEJEZ,2013-04-30,2002-05-01
3,73975,H,1941-10-16,NaT,2009-10-01,558,1.075000,1.075000,504.06,504.06,541.86,VEJEZ,2009-09-30,2004-10-01
4,13549,H,1936-08-23,NaT,2009-10-01,489,1.000000,1.000000,371.75,371.75,371.75,VEJEZ,2009-09-30,2004-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627473,16037539,H,1953-04-22,NaT,1998-09-01,314,0.702083,0.702083,471.02,471.02,214.66,VEJEZ,2016-07-31,2011-08-01
627474,16048044,H,1952-12-12,NaT,2019-08-01,505,1.025000,1.025000,385.78,385.78,790.84,VEJEZ,2019-07-31,2014-08-01
627475,15947582,H,1952-12-13,NaT,2013-11-01,405,0.787500,0.787500,647.15,647.15,1074.19,VEJEZ,2020-03-31,2011-05-01
627476,16134814,H,1955-09-24,NaT,2018-01-01,509,1.025000,1.025000,1621.07,1621.07,3323.20,VEJEZ,2017-12-31,2012-01-01


In [39]:
afi = pd.read_csv(ruta_afi, delimiter='\t', encoding='utf-8', decimal='.')

In [40]:
afi = afi.set_index(['CEDULA_COD'])

In [41]:
afi

,ANIO,MES,SALARIO,PORCENTAJE_AP,SECTOR
CEDULA_COD,,,,,
20057738,2003,3,135.03,20.5,PRIVADO
15613923,2003,11,65.61,20.5,PRIVADO
16057398,1996,11,56.65,20.5,PRIVADO
16023482,1997,9,15.11,20.5,PRIVADO
16021285,1998,1,27.27,20.5,PUBLICO
...,...,...,...,...,...
16773012,2005,9,230.00,20.5,PRIVADO
5709849,2007,1,278.95,30.5,PUBLICO
19960695,2008,9,1400.00,20.5,PRIVADO


In [42]:
afi.unstack(2)

        CEDULA_COD
ANIO    20057738         2003
        15613923         2003
        16057398         1996
        16023482         1997
        16021285         1998
                       ...   
SECTOR  16773012      PRIVADO
        5709849       PUBLICO
        19960695      PRIVADO
        11118795      PRIVADO
        16263858      PRIVADO
Length: 332776020, dtype: object